In [1]:
#6-frame translation

from Bio.Seq import Seq
from Bio import SeqIO
for seq_record in SeqIO.parse('combined-named.fasta','fasta'):
    #forward 3 translations
    fwd1 = seq_record.seq.translate()
    fwd2 = seq_record.seq[1:].translate()
    fwd3 = seq_record.seq[2:].translate()
    revcomp = seq_record.seq.reverse_complement()
    rev1 = revcomp.translate()
    rev2 = revcomp[1:].translate()
    rev3 = revcomp[2:].translate()
    with open('6_frame_translate.fasta','a') as file_handle:
        file_handle.write('>' + seq_record.description + '_1' + '\n' + str(fwd1) + '\n')
        file_handle.write('>' + seq_record.description + '_2' + '\n' + str(fwd2) + '\n')
        file_handle.write('>' + seq_record.description + '_3' + '\n' + str(fwd3) + '\n')
        file_handle.write('>' + seq_record.description + '_4' + '\n' + str(rev1) + '\n')
        file_handle.write('>' + seq_record.description + '_5' + '\n' + str(rev2) + '\n')
        file_handle.write('>' + seq_record.description + '_6' + '\n' + str(rev3) + '\n')

/home/yy/.local/lib/python3.8/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [3]:
#search HMM models
import subprocess

conserved_domains = ['CC2-LZ','Gag-MA','Gag-Capsid','Env-GP41','Pol-MLVIN-C','RL40e','Pol-rve','SH2','SH3','Env-TLV-coat','zf-C2H2',\
    'Cornifin','Sina','SOCS','Peptidase','Epiglycanin','Tnp_22','Transposase_22','KOW']

def hmmsearch(domain):
    subprocess.run('hmmsearch {}.hmm 6_frame_translate.fasta > {}_output.txt'.format(domain, domain), shell = True)

for domain in conserved_domains:
    hmmsearch(domain)

In [4]:
#parse trees - identify the names

border_chars = [',','(',')',':']
with open('tree.nwk') as file:
    treelist = file.readlines()
tree = treelist[0]
resultlist = []
finallist = []

def extend(index):  #extends the word
    if index < len(tree):
        if tree[index] in border_chars and tree[index+1] not in border_chars:
            return True
        else:
            return False
    else:
        return False

def evaluate(index):    #evaluates if the end of this word is reached. Returns False if the end is reached
    if index < len(tree) and tree[index] in border_chars:
        return True
    else:
        return False

#extract everything between border characters as names
for index in range(len(tree)):
    if extend(index):
        orig_index = index
        if index < 17000:
            while True:
                eval = evaluate(index+1)
                index += 1
                if eval:
                    resultlist.append(tree[orig_index+1:index])
                    break
        else:
            pass

#filter unrelated sequences out of resultlist
for element in resultlist:
    if element[0:2] == 'NW':
        finallist.append(element)

#parse HMMER output
from Bio import SearchIO
for domain in conserved_domains:
    filename = '{}_output.txt'.format(domain)
    qresult = list(SearchIO.parse(filename, 'hmmer3-text'))
    result = qresult[-1]
    for hit in result:        
        try:
            if hit.evalue < 0.0001 and finallist.index(hit.id[:-2]):    #e-value and element in the list conditions
                index = tree.find(hit.id[0])
                orig_index = index  #isolate the current string of the element
                while True:
                    eval = evaluate(index+1)
                    index += 1
                    if eval:
                        if domain not in tree[orig_index+1:index]:
                            newname = hit.id[:-2] + '_{}'.format(domain)
                            tree = tree.replace(hit.id[:-2], newname)
                        break
        except ValueError:
            pass
with open('marked_tree.nwk','w') as file_handle:
    file_handle.write(tree)